In [1]:
import os
# データのダウンロード
path = "Hazumi1712"
if(not os.path.exists(path)):
    !git clone https://github.com/ouktlab/Hazumi1712.git
    !git clone https://github.com/ouktlab/Hazumi1902.git
    !git clone https://github.com/ouktlab/Hazumi1911.git

In [2]:
# パッケージのインポート
import glob
import pandas as pd
import numpy as np

import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from keras.utils import plot_model

from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.models import load_model

import numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

from sklearn.preprocessing import StandardScaler
import statistics

from scipy.io import arff

import datetime


In [3]:
class dataset_extractor:
    '''初期化'''
    def __init__(self, version = '1712'):
        self.path = 'Hazumi{0}/dumpfiles/'.format(version)
        self.files = glob.glob(self.path+'*')
        self.list_p = []
        self.list_w = []
        self.list_f = []
        self.list_m = []
        self.list_t = []
        self.train_feature = pd.DataFrame()
        self.train_label = pd.DataFrame()
        self.test_feature = pd.DataFrame()
        self.test_label = pd.DataFrame()
        self.split_pandas_list()
        self.connect_feature_Hazumi = []
        self.connect_label_Hazumi = []
        self.scaler_Hazumi = StandardScaler()
        self.make_connect_data()
        self.standardize_data()
    '''Hazumiのデータ抽出''' 
    def one_data_extractor(self, file):
        return pd.read_csv(file)
    '''データを抽出'''
    def extract_data(self, df, start_name='pcm_RMSenergy_sma_max', end_name='F0_sma_de_kurtosis'):
        extract_df = df.loc[:, start_name:end_name]
        return extract_df
    '''特徴量（韻律）の抽出'''
    def extract_prosody(self, df):
        prosody_df = self.extract_data(df=df, start_name='pcm_RMSenergy_sma_max', end_name='F0_sma_de_kurtosis')
        return prosody_df
    '''特徴量（wordvec）の抽出'''
    def extract_wordvec(self, df):
        wordvec_df = self.extract_data(df=df, start_name='word#001', end_name='word#615')
        return wordvec_df
    '''特徴量（顔表情特徴量）の抽出'''
    def extract_face(self, df):
        face_df = self.extract_data(df=df, start_name='17_acceleration_max', end_name='AU45_c_mean')
        return face_df
    '''特徴量（動作特徴量）の抽出'''
    def extract_move(self, df):
        move_df = self.extract_data(df=df, start_name='HandLeft_acceleration_max', end_name='duration')
        return move_df
    '''アノテーション（ternaryとなっているもの）'''
    def extract_ternary(self, df):
        # ternary_df = self.extract_data(df=df, start_name='IN_ternary', end_name='TS_ternary')
        ternary_df = self.extract_data(df=df, start_name='IN_ternary', end_name='IN_ternary')
        return ternary_df
    '''特徴量とラベルに分割'''
    def split_pandas(self, df):
        df_p = self.extract_prosody(df=df)
        df_w = self.extract_wordvec(df=df)
        df_f = self.extract_face(df=df)
        df_m = self.extract_move(df=df)
        df_t = self.extract_ternary(df=df)
        return df_p,df_w,df_f,df_m,df_t
    '''特徴量とラベルの集合体にする'''
    def split_pandas_list(self):
        for f in self.files:
            df = self.one_data_extractor(f)
            df_p, df_w,df_f, df_m, df_t = self.split_pandas(df)
            self.list_p.append(df_p.values)
            self.list_w.append(df_w.values)
            self.list_f.append(df_f.values)
            self.list_m.append(df_m.values)
            self.list_t.append(df_t.values)
    '''指定した1人のデータを訓練データに、その他のデータをテストデータにする'''
    def assign_train_test_Hazumi(self, num):
        self.test_feature = self.list_p[num]
        self.test_label = self.list_t[num]
        self.train_feature = []
        self.train_label = []
        for n in range(len(self.list_p)):
            if(n != num):
                self.train_feature.extend(self.list_p[n])
                self.train_label .extend(self.list_t[n].ravel())
                # self.train_feature.extend() = pd.concat([self.train_feature, self.list_p[n]])
                # self.train_label = pd.concat([self.train_label, self.list_t[n]])
        self.train_feature = np.array(self.train_feature)
        self.train_label = np.array(self.train_label).ravel()
    '''全部つなぎのデータを取ってくる'''
    def make_connect_data(self):
        for n in range(len(self.files)):
            self.connect_feature_Hazumi.extend(self.list_p[n])
            self.connect_label_Hazumi.extend(self.list_t[n])
        self.connect_feature_Hazumi = np.array(self.connect_feature_Hazumi)
        self.connect_label_Hazumi = np.array(self.connect_label_Hazumi).ravel()
    '''データを標準化する'''
    def standardize_data(self):
        self.scaler_Hazumi.fit(self.connect_feature_Hazumi)
        for n in range(len(self.files)):
            self.list_p[n] = self.scaler_Hazumi.transform(self.list_p[n])
        
        self.connect_feature_Hazumi = self.scaler_Hazumi.transform(self.connect_feature_Hazumi)
        
extractor = dataset_extractor()
print(len(extractor.connect_feature_Hazumi))

2261


In [4]:
class SVM_model:
    def __init__(self):
        self.clf = svm.SVC(gamma="scale")
    def fit(self, features, labels):
        self.clf.fit(features, labels)
    def predict(self, features):
        return self.clf.predict(features)

In [5]:
class KNN_model:
    def __init__(self, K=3):
        self.clf = KNeighborsClassifier(n_neighbors=K)
    def fit(self, features, labels):
        self.clf.fit(features, labels)
    def predict(self, features):
        return self.clf.predict(features)

In [7]:
class DNN_model:
    def __init__(self):
        input_size = 384
        output_size = 3
        # Sequentialモデルの場合
        model = Sequential()
        model.add(Dense(input_size//2, activation='sigmoid', input_dim=input_size))
        model.add(Dense(input_size//4, activation='sigmoid'))
        model.add(Dense(input_size//8, activation='sigmoid'))
        model.add(Dense(output_size, activation='softmax'))
        middle_model = Model(inputs=model.input, outputs=model.layers[2].output)
        model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
        self.clf = model
        self.history = None
        self.epochs = 80
        self.batch_size = 32
    def fit(self, features, labels):
        labels = to_categorical(labels)
        history = self.clf.fit(features, labels, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return history
    def predict(self, features):
#         print(np.argmax(self.clf.predict(features)))
        return self.clf.predict(features).argmax(axis=1)

In [8]:
class AggloCluster_model:
    def __init__(self):
        N = 2
        self.clf = AgglomerativeClustering(n_clusters = 2)
    def fit(self, features):
        self.clf.fit(features)
    def predict(self, features):
        return self.clf.predict(features)

In [9]:
class DBSCAN_model:
    def __init__(self):
        self.clf = DBSCAN()
    def fit(self, features):
        self.clf.fit(features)
    def predict(self, features):
        return self.clf.predict(features)

In [10]:
class Kmeans_model:
    def __init__(self, n_clusters=2):
        self.clf = KMeans(n_clusters=n_clusters)
    def fit(self, features):
        self.clf.fit(features)
    def predict(self, features):
        return self.clf.fit(features)
    def fit_predict(self, features):
        return self.clf.fit_predict(features)

In [11]:
def cross_validation(mode="KNN"):
    result_train = []
    result_test = []
    datasets = dataset_extractor()
    for num in range(len(datasets.files)):
        if(mode == "DNN"):
            model = DNN_model()
            pass
        elif(mode == "SVM"):
            model = SVM_model()
            pass
        elif(mode == "KNN"):
            model = KNN_model()
            pass
        datasets.assign_train_test_Hazumi(num)
#         model.fit(datasets.train_feature.values, datasets.train_label.values.ravel())
#         predict_train = model.predict(datasets.train_feature.values)
#         predict_test = model.predict(datasets.test_feature.values)
#         result_train.append(accuracy_score(predict_train, datasets.train_label.values.ravel()))
#         result_test.append(accuracy_score(predict_test, datasets.test_label.values.ravel()))
        model.fit(datasets.train_feature, datasets.train_label.ravel())
        predict_train = model.predict(datasets.train_feature)
        predict_test = model.predict(datasets.test_feature)
        result_train.append(accuracy_score(predict_train, datasets.train_label.ravel()))
        result_test.append(accuracy_score(predict_test, datasets.test_label.ravel()))
    print("mode:{0}, acc of...".format(mode))
    print("train:{0}, test:{1}".format(statistics.mean(result_train), statistics.mean(result_test)))
    
cross_validation(mode="KNN")
cross_validation(mode="SVM")
cross_validation(mode="DNN")

mode:KNN, acc of...
train:0.7230965663006164, test:0.45050499987792825
mode:SVM, acc of...
train:0.7431213387957137, test:0.5290102467303135
mode:DNN, acc of...
train:0.9999148177899287, test:0.49051888844001773


In [12]:
"""
NMIもしくはARIで精度を出力する
"""
def non_teacher_accuracy():
    datasets = dataset_extractor()
    model = Kmeans_model(n_clusters=2)
    Hazumi_predicts = model.fit_predict(datasets.connect_feature_Hazumi)
    print("Hazumi_ARI:", adjusted_rand_score(Hazumi_predicts, datasets.connect_label_Hazumi))
    print("Hazumi_NMI:", normalized_mutual_info_score(Hazumi_predicts, datasets.connect_label_Hazumi))
non_teacher_accuracy()

Hazumi_ARI: 0.04133434087083038
Hazumi_NMI: 0.044312960523494836
